In [1]:
import requests, json, os, sys, time, re
from datetime import datetime
from multiprocessing import Pool
from sqlalchemy import *

In [2]:
def split_list(lst_long,n):
    lst_splitted = []
    if len(lst_long) % n == 0:
        totalBatches = len(lst_long) / n
    else:
        totalBatches = len(lst_long) / n + 1
    for i in xrange(totalBatches):
        lst_short = lst_long[i*n:(i+1)*n]
        lst_splitted.append(lst_short)
    return lst_splitted


def show_work_status(singleCount, totalCount, currentCount=0):
    currentCount += singleCount
    percentage = 1. * currentCount / totalCount * 100
    status =  '>' * int(percentage)  + ' ' * (100 - int(percentage))
    sys.stdout.write('\rStatus: [{0}] {1:.2f}% '.format(status, percentage))
    sys.stdout.flush()
    if percentage >= 100:
        print('\n')

In [3]:
path_user_id = 'steam_user_id.txt'
with open(path_user_id, 'rb') as f:
    lst_user_id = f.readlines()[:100]

In [4]:
def worker(lst_user_id_temp):
    dic_temp = {}
    for user_id in lst_user_id_temp:
        base_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
        params = {
            'key' : 'D0C62157A8941F12A687382B6D635449',
            'steamid' : user_id.strip(),
            'format' : 'json' }
        r = requests.get(base_url, params = params)
        user_inventory = r.json().get('response').get('games')
        dic_temp.update({user_id.strip():user_inventory})
        time.sleep(.5)
    return dic_temp

In [5]:
p = Pool(2)

In [6]:
total_count = len(lst_user_id)

In [7]:
current_count = 0

In [8]:
show_work_status(0, total_count, current_count)

Status: [                                                                                                    ] 0.00% 

In [ ]:
dic_master = {}
for i in split_list(lst_user_id,50):
    lst_temp_dic = p.map(worker, split_list(i,25))
    for j in lst_temp_dic:
        dic_master.update(j)
    show_work_status(len(i), total_count, current_count)
    current_count += len(i)
    time.sleep(5)

In [ ]:
p = Pool(2)

total_count = len(lst_user_id)
current_count = 0
show_work_status(0, total_count, current_count)

dic_master = {}
for i in split_list(lst_user_id,50):
    lst_temp_dic = p.map(worker, split_list(i,25))
    for j in lst_temp_dic:
        dic_master.update(j)
    show_work_status(len(i), total_count, current_count)
    current_count += len(i)
    time.sleep(5)

Status: [                                                                                                    ] 0.00% 

In [9]:
with open('data/crawled_user_inventory.txt', 'wb') as f:
    for user_id, user_inventory in dic_master.items():
        f.write(json.dumps({user_id:user_inventory}))
        f.write('\n')

In [ ]:
# steam app info

In [10]:
r = requests.get('http://steamspy.com/api.php?request=all')
dic_app_user = r.json()

In [11]:
len(dic_app_user)

14475

In [13]:
path_app_detail = 'data/app_detail.txt'

In [12]:
lst_app_id = dic_app_user.keys()
total_count = len(lst_app_id)
current_count = 0
show_work_status(0, total_count, current_count)


with open(path_app_detail, 'wb') as f:
    for app_id in lst_app_id:
        url_app_detail = ('http://store.steampowered.com/api/appdetails?appids=%s') % (app_id)
        for i in range(3):
            try:
                r = requests.get(url_app_detail)
                result = r.json()
                break
            except:
                time.sleep(5)
                pass
        f.write(json.dumps(result))
        f.write('\n')
        show_work_status(1, total_count, current_count)
        current_count += 1
        if current_count % 200 == 0:
            time.sleep(300)
        else:
            time.sleep(.5)

Status: [                                                                                                    ] 0.00% 

NameError: name 'path_app_info' is not defined

In [14]:
with open(path_app_detail, 'r') as f:
    dic_steam_app = {'initial_price':{},'name':{},'score':{},'windows':{},'mac':{},'linux':{},'type':{},'release_date':{},'recommendation':{},'header_image':{}}
    lst_raw_string = f.readlines()[:3000]
    total_count = len(lst_raw_string)
    current_count = 0
    show_work_status(0, total_count, current_count)
    for raw_string in lst_raw_string:
        app_data = json.loads(raw_string).values()[0]
        if app_data != {}:
            steam_id = app_data.get('steam_appid')
            initial_price = app_data.get('price_overview',{}).get('initial')
            if app_data.get('is_free') == True:
                initial_price = 0
            app_name = app_data.get('name')
            critic_score = app_data.get('metacritic', {}).get('score')
            app_type = app_data.get('type')
            for (platform, is_supported) in app_data.get('platforms',{}).items():
                if is_supported == True:
                    dic_steam_app[platform].update({steam_id:1})
            if app_data.get('release_date',{}).get('coming_soon') == False:
                release_date = app_data.get('release_date',{}).get('date')
                if not release_date == '':
                    try:
                        release_date = datetime.strptime(release_date, '%b %d, %Y')
                    except:
                        try:
                            release_date = datetime.strptime(release_date, '%d %b, %Y')
                        except:
                            release_date = None
                    

            recommendation = app_data.get('recommendations',{}).get('total')
            header_image = app_data.get('header_image')
            dic_steam_app['initial_price'].update({steam_id:initial_price})
            dic_steam_app['name'].update({steam_id:app_name})
            dic_steam_app['score'].update({steam_id:critic_score})
            dic_steam_app['type'].update({steam_id:app_type})
            dic_steam_app['release_date'].update({steam_id:release_date})
            dic_steam_app['recommendation'].update({steam_id:recommendation})
            dic_steam_app['header_image'].update({steam_id:header_image})
        show_work_status(1, total_count, current_count)
        current_count += 1

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [15]:
import pandas as pd
df_app_info = pd.DataFrame(dic_steam_app)

In [16]:
df_app_info.head()

,header_image,initial_price,linux,mac,name,recommendation,release_date,score,type,windows
60,http://cdn.akamai.steamstatic.com/steam/apps/6...,499,1.0,1.0,Ricochet,2109.0,2000-11-01,NaN,game,1
300,http://cdn.akamai.steamstatic.com/steam/apps/3...,999,1.0,1.0,Day of Defeat: Source,7753.0,2010-07-12,80.0,game,1
550,http://cdn.akamai.steamstatic.com/steam/apps/5...,1999,1.0,1.0,Left 4 Dead 2,159088.0,2009-11-16,89.0,game,1
1300,http://cdn.akamai.steamstatic.com/steam/apps/1...,999,NaN,NaN,SiN Episodes: Emergence,408.0,2006-05-10,75.0,game,1
1640,http://cdn.akamai.steamstatic.com/steam/apps/1...,649,NaN,NaN,Disciples II: Gallean's Return,448.0,2006-07-06,84.0,game,1


In [22]:
engine = create_engine('mysql+pymysql://:@127.0.0.1/game_recommendation?charset=utf8mb4')
df_app_info.to_sql('tbl_app_info_test',engine, if_exists='replace')